# Spooky Author Identification
[連結](https://www.kaggle.com/c/spooky-author-identification/submissions?sortBy=date&group=all&page=1)
---

In [1]:
import tensorflow as tf
import numpy as np
import random

In [ ]:
# 為了讓設定是可以 重複製造
random.seed(1)
tf.set_random_seed(1)

In [ ]:
# 字典檔
WordDictionary = []
PeopleDictionary = ["EAP", "HPL", "MWS"]

In [ ]:
# 把垃圾字元去掉
def deleteNotWord(line):
    line = line.replace(",", "")
    line = line.replace(";", "")
    line = line.replace("\'", "")
    line = line.replace("\"", "")
    return line

In [ ]:
# 根據單字，找在字典裏面的哪裡
def findIndexInWordDictionary(word):
    word = word.lower()
    for i in range(0, len(WordDictionary)):
        # 如果有找到的話
        if(word == WordDictionary[i]):
            return i
        
    # 沒找到
    return -1

In [ ]:
# 找 People Dictionary 裡面，名字在哪個 index
def findIndexInPeopleDictionary(name):
    outputArray = [0] * len(PeopleDictionary)
    for i in range(len(PeopleDictionary)):
        if(PeopleDictionary[i] == name):
            outputArray[i] = 1
    return outputArray

In [ ]:
def passAndAddWord(line):
    words = line.split(" ")
    for i in range(0, len(words)):
        if(findIndexInWordDictionary(words[i]) == -1):
            WordDictionary.append(words[i])

In [ ]:
# 讀檔
trainData = open("./Data/train.csv", "r")

# 要存出去的部分
TextList = []
AuthorList = []

# 吐掉第一行
trainData.readline()

# 將檔案做處理
for line in trainData:
    # 依照 "," 分，並把最前面的雙引號及後面去掉
    lineData = line.split("\",\"")
    lineData[0] = lineData[0][1:]
    lineData[2] = lineData[2][0:len(lineData[2]) - 2]
    
    lineData[1] = deleteNotWord(lineData[1])
    passAndAddWord(lineData[1])
    
    # 加進 List 供以後存取
    TextList.append(lineData[1])
    AuthorList.append(lineData[2])

# 測試字典檔大小
DictionarySize = len(WordDictionary)
print(DictionarySize)

In [ ]:
# 關閉檔案
trainData.close()

In [ ]:
totalFileSize = 19579
testFileSize = int(totalFileSize * 0.01)
trainFileSize = totalFileSize - testFileSize
print("Total File Size: " + format(totalFileSize))
print("Train File Size: " + format(trainFileSize))
print("Test File Size: " + format(testFileSize))

In [ ]:
# 取出 Test 的 Index
TestIndex = random.sample(range(totalFileSize), testFileSize)

In [ ]:
# 判斷是不是在 Test Index 裡面
def findIndexInTestIndex(index):
    for i in range(0, len(TestIndex)):
        if(TestIndex[i] == index):
            return True
    return False

In [ ]:
# 判斷字在不在字典裏面，並 Output 有的數目
def CountWordInDictionary(line):
    outputArray = [0] * DictionarySize
    
    lineData = line.split(" ")
    for word in lineData:
        index = findIndexInWordDictionary(word)
        outputArray[index] += 1
    
    wordCount = len(lineData)
    for i in range(0, DictionarySize):
        outputArray[i] /= wordCount
    return outputArray

In [ ]:
# 要丟進 Tensorflow 學習的參數
# Train
TrainInputList = []
TrainOutputList = []
# Test
TestInputList = []
TestOutputList = []
for i in range(0, len(TextList)):
    # 先處理完第 i 筆資料
    TempInput = CountWordInDictionary(TextList[i])
    TempOutput = findIndexInPeopleDictionary(AuthorList[i])
    
    # 判斷要加到哪裡
    if(not findIndexInTestIndex(i)):
        TrainInputList.append(TempInput)
        TrainOutputList.append(TempOutput)
    else:
        TestInputList.append(TempInput)
        TestOutputList.append(TempOutput)

## 開始建構 Tensorflow

---

In [ ]:
layer1_size = 10000
layer2_size = 300

In [ ]:
# 輸入
inputWordProb = tf.placeholder(tf.float32, [None, len(WordDictionary)], name = "InputProb")

# 輸出
outputProb = tf.placeholder(tf.float32,[None, len(PeopleDictionary)], name = "OutputProb")
  
# 初始化
weghtInit = tf.random_normal_initializer(mean = 0, stddev= = 0.3)
biasInit = tf.random_normal_initializer(mean = 0, stddev = 0.1)

# Layer 1
with tf.name_scope("layer1"):
    layer1 = tf.layers.dense(
        inputs = inputWordProb,
        units = layer1_size,
        activation = tf.nn.relu,
        kernel_initializer = weghtInit,
        bias_initializer = bias_initializer
    )